In [3]:
groq_api_key = "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"

In [4]:
import openai

client = openai.OpenAI(
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1"
)

In [9]:
response = client.chat.completions.create(
    model="groq/compound",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello, just testing the Groq API connection!"}
    ]
)
print(response.choices[0].message.content)


Hello! I'm here and ready to help. The Groq API connection is looking good! What's on your mind? Want to test with a question or task?


In [10]:
# Initialize empty conversation history list
conversation_history = []

# Function to add a message (user or assistant)
def add_message(role, content):
    conversation_history.append({"role": role, "content": content})

# Add sample user and assistant messages
add_message("user", "Hello!")
add_message("assistant", "Hi! How can I help you today?")

# Print the current conversation
print(conversation_history)

[{'role': 'user', 'content': 'Hello!'}, {'role': 'assistant', 'content': 'Hi! How can I help you today?'}]


In [11]:
def truncate_by_turns(n):
    return conversation_history[-n:]

# Example: keep only last 1 turn
print(truncate_by_turns(1))

[{'role': 'assistant', 'content': 'Hi! How can I help you today?'}]


In [12]:
def truncate_by_chars(max_chars):
    truncated = []
    total_chars = 0
    # Iterate backwards until max_chars reached
    for msg in reversed(conversation_history):
        msg_len = len(msg["content"])
        if total_chars + msg_len > max_chars:
            break
        truncated.insert(0, msg)  # Insert at front
        total_chars += msg_len
    return truncated

# Example: keep messages totaling max 10 chars
print(truncate_by_chars(10))

[]


In [15]:
def summarize_conversation(conversation):
    # Prepare the prompt with conversation content
    text_to_summarize = "\n".join([f"{msg['role']}: {msg['content']}" for msg in conversation])
    prompt = f"Summarize the following conversation briefly:\n\n{text_to_summarize}"

    # Call Groq API for completion
    response = client.chat.completions.create(
        model="groq/compound",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    summary = response.choices[0].message.content
    return summary

# Example usage:
summary_text = summarize_conversation(conversation_history)
print("Summary:\n", summary_text)


Summary:
 The conversation was a brief greeting, with the user saying "Hello!" and the assistant responding with "Hi! How can I help you today?" There was no further discussion.


In [16]:
# Initialize conversation history and a counter
conversation_history = []
message_count = 0
k = 3  # summarization frequency

def add_message(role, content):
    global message_count, conversation_history
    conversation_history.append({"role": role, "content": content})
    message_count += 1

    # After every k messages, summarize and replace history
    if message_count % k == 0:
        summary = summarize_conversation(conversation_history)
        conversation_history = [{"role": "system", "content": "Summary of previous conversation:"},
                                {"role": "assistant", "content": summary}]
        print("Conversation summarized and updated.")

def print_history():
    for msg in conversation_history:
        print(f"{msg['role'].capitalize()}: {msg['content']}")

# Test by adding messages
add_message("user", "Hello!")
add_message("assistant", "Hi! How can I help you today?")
add_message("user", "Can you help me with the assignment?")
print_history()


Conversation summarized and updated.
System: Summary of previous conversation:
Assistant: I'm here to assist you with your assignment. What's the assignment about and what specifically do you need help with?


In [17]:
# Truncate by last n turns
def truncate_by_turns(history, n):
    return history[-n:]

# Truncate by max characters (approximate, counting message content length)
def truncate_by_chars(history, max_chars):
    truncated = []
    total_chars = 0
    for msg in reversed(history):
        msg_len = len(msg["content"])
        if total_chars + msg_len > max_chars:
            break
        truncated.insert(0, msg)
        total_chars += msg_len
    return truncated

# Updated add_message to accept truncation parameters
def add_message_with_truncation(role, content, max_turns=None, max_chars=None):
    global message_count, conversation_history
    conversation_history.append({"role": role, "content": content})
    message_count += 1

    # Apply truncation before summarizing
    truncated_history = conversation_history
    if max_turns:
        truncated_history = truncate_by_turns(truncated_history, max_turns)
    if max_chars:
        truncated_history = truncate_by_chars(truncated_history, max_chars)

    # After every k messages, summarize and replace history with summary + truncated
    if message_count % k == 0:
        summary = summarize_conversation(truncated_history)
        conversation_history = [{"role": "system", "content": "Summary of previous conversation:"},
                                {"role": "assistant", "content": summary}]
        print("Conversation summarized and updated.")
    else:
        conversation_history = truncated_history

def print_history():
    for msg in conversation_history:
        print(f"{msg['role'].capitalize()}: {msg['content']}")

# Test example:
add_message_with_truncation("user", "Hello!", max_turns=3, max_chars=100)
add_message_with_truncation("assistant", "Hi! How can I help you today?", max_turns=3, max_chars=100)
add_message_with_truncation("user", "Can you help me with the assignment?", max_turns=3, max_chars=100)

print_history()

Conversation summarized and updated.
System: Summary of previous conversation:
Assistant: I'm here to help. What specifically do you need help with regarding your assignment?


In [18]:
# Clear conversation history and reset counter for fresh tests
conversation_history = []
message_count = 0

# Define test messages
test_messages = [
    ("user", "Hello!"),
    ("assistant", "Hi! How can I assist you today?"),
    ("user", "I need help with my AI assignment."),
    ("assistant", "Sure! What part do you want to start with?"),
    ("user", "Let’s begin with conversation management."),
    ("assistant", "Great! I can help you with that."),
]

# Test 1: Truncate by last 2 turns
print("Test 1: Truncate by last 2 turns")
for role, content in test_messages:
    add_message_with_truncation(role, content, max_turns=2)
print_history()
print("\n")

# Clear for next test
conversation_history = []
message_count = 0

# Test 2: Truncate by max 50 characters
print("Test 2: Truncate by max 50 characters")
for role, content in test_messages:
    add_message_with_truncation(role, content, max_chars=50)
print_history()
print("\n")

# Clear for next test
conversation_history = []
message_count = 0

# Test 3: Truncate by last 3 turns and max 100 characters
print("Test 3: Truncate by last 3 turns and max 100 characters")
for role, content in test_messages:
    add_message_with_truncation(role, content, max_turns=3, max_chars=100)
print_history()


Test 1: Truncate by last 2 turns
Conversation summarized and updated.
Conversation summarized and updated.
System: Summary of previous conversation:
Assistant: The conversation between the user and the assistant is about starting a discussion on conversation management. The user initiated the conversation, and the assistant responded positively, offering help. That's the summary of the conversation.


Test 2: Truncate by max 50 characters
Conversation summarized and updated.
Conversation summarized and updated.
System: Summary of previous conversation:
Assistant: There is no conversation to summarize. This conversation just started. What would you like to talk about? I can summarize our conversation at the end if you'd like.


Test 3: Truncate by last 3 turns and max 100 characters
Conversation summarized and updated.
Conversation summarized and updated.
System: Summary of previous conversation:
Assistant: The conversation between the user and the assistant is about conversation manage

# Task 1: Conversation Management & Summarization

*(All code and outputs related to conversation history, truncation, and summarization above)*

---

# Task 2: JSON Schema Classification & Extraction

*(Starting below will be the code and explanations for structured info extraction from chats)*


In [19]:
# Define JSON schema for extraction
json_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "Full name of the user"},
        "email": {"type": "string", "description": "User's email address"},
        "phone": {"type": "string", "description": "User's phone number"},
        "location": {"type": "string", "description": "User's location or city"},
        "age": {"type": "integer", "description": "User's age in years"}
    },
    "required": ["name", "email", "phone", "location", "age"]
}

# Sample chat conversations to parse
sample_chats = [
    "Hi, my name is Alice Johnson. You can reach me at alice.j@example.com or call me on 123-456-7890. I live in Bangalore and I am 24 years old.",
    "Hello, this is Bob Smith. My email is bobsmith@mail.com and phone number is (555) 987 6543. I currently reside in Mumbai. Age is 30.",
    "Hey! I'm Carol, living in Delhi. Email carol_xyz@gmail.com, phone 9876543210. I'm 28 years old."
]


In [35]:
import re
import json


def clean_json_text(raw_text):
    # Remove triple backticks and optional 'json' tag safely by splitting string
    text = re.sub(r"```" + r"(?:json)?", "", raw_text, flags=re.DOTALL).strip()
    # Remove any remaining triple backticks
    text = text.replace("```", "")
    # Remove markdown mailto links embedded inside quoted JSON strings:
    text = re.sub(r'"$$([^$$]+)$$$$mailto:[^)]+$$"', r'"\1"', text)
    return text


def extract_info_from_chat_clean(chat_text):
    prompt = (
        f"Extract the following details from the chat as valid JSON matching this schema:\n"
        f"{json.dumps(json_schema, indent=2)}\n\n"
        f"Chat:\n{chat_text}\n\n"
        f"Return ONLY the JSON object."
    )

    response = client.chat.completions.create(
        model="groq/compound",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that extracts user info as JSON."},
            {"role": "user", "content": prompt}
        ]
    )
    raw_output = response.choices[0].message.content
    cleaned_output = clean_json_text(raw_output)
    try:
        data = json.loads(cleaned_output)
        data = fix_emails(data)
        return data
    except Exception as e:
        print("Error parsing JSON after cleaning:", e)
        print("Cleaned output was:\n", cleaned_output)
        return None

# Test extraction on all sample chats
for i, chat in enumerate(sample_chats, 1):
    print(f"Chat {i}:")
    extracted_data = extract_info_from_chat_clean(chat)
    print(extracted_data)
    print("\n")

Chat 1:
{'name': 'Alice Johnson', 'email': 'alice.j@example.com', 'phone': '123-456-7890', 'location': 'Bangalore', 'age': 24}


Chat 2:
{'name': 'Bob Smith', 'email': 'bobsmith@mail.com', 'phone': '(555) 987 6543', 'location': 'Mumbai', 'age': 30}


Chat 3:
{'name': 'Carol', 'email': 'carol_xyz@gmail.com', 'phone': '9876543210', 'location': 'Delhi', 'age': 28}




In [49]:
import re
import json

def clean_json_text(raw_text):
    # Remove triple backticks and optional 'json' tag safely by splitting string
    text = re.sub(r"```" + r"(?:json)?", "", raw_text, flags=re.DOTALL).strip()
    # Remove any remaining triple backticks
    text = text.replace("```", "")
    # Remove markdown mailto links embedded inside quoted JSON strings:
    text = re.sub(r'"$$([^$$]+)$$$$mailto:[^)]+$$"', r'"\1"', text)
    return text

def normalize_nulls(data_dict):
    for key, value in data_dict.items():
        if value in ('null', None, 'unknown', 0, ''):
            data_dict[key] = None
    return data_dict

def extract_info_from_chat_clean(chat_text):
    prompt = (
        f"Extract the following details from the chat as valid JSON matching this schema:\n"
        f"{json.dumps(json_schema, indent=2)}\n\n"
        f"Chat:\n{chat_text}\n\n"
        f"Return ONLY the JSON object."
    )

    response = client.chat.completions.create(
        model="groq/compound",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that extracts user info as JSON."},
            {"role": "user", "content": prompt}
        ]
    )
    raw_output = response.choices[0].message.content
    cleaned_output = clean_json_text(raw_output)

    # Gracefully handle empty or invalid JSON
    if not cleaned_output.strip() or cleaned_output.strip() == '{}':
        return None

    try:
        data = json.loads(cleaned_output)
        return data
    except Exception as e:
        print("Error parsing JSON after cleaning:", e)
        print("Cleaned output was:\n", cleaned_output)
        return None

# Extended chats for testing
extended_chats = sample_chats + [
    "Hey, I'm Mike. Email mike123@mail.com. Location is Hyderabad.",
    "My name's Sara. Phone: 555-123-4567. Age 22. Email sara@example.com.",
    "Hello, no contact info here, just saying hi!",
    "This is David. Email: david@example.com, lives in Pune. Age unknown."
]

# Run extraction, normalization, and print results
print("Running extended chat extraction tests with full cleaning and normalization:\n")

for i, chat in enumerate(extended_chats, 1):
    print(f"Extended Chat {i}:")
    raw_data = extract_info_from_chat_clean(chat)
    if raw_data:
        normalized_data = normalize_nulls(raw_data)
        print(normalized_data)
    else:
        print("Failed to extract data or no info found.")
    print("\n")


Running extended chat extraction tests with full cleaning and normalization:

Extended Chat 1:
{'name': 'Alice Johnson', 'email': 'alice.j@example.com', 'phone': '123-456-7890', 'location': 'Bangalore', 'age': 24}


Extended Chat 2:
{'name': 'Bob Smith', 'email': 'bobsmith@mail.com', 'phone': '(555) 987 6543', 'location': 'Mumbai', 'age': 30}


Extended Chat 3:
{'name': 'Carol', 'email': 'carol_xyz@gmail.com', 'phone': '9876543210', 'location': 'Delhi', 'age': 28}


Extended Chat 4:
{'name': 'Mike', 'email': 'mike123@mail.com', 'phone': None, 'location': 'Hyderabad', 'age': None}


Extended Chat 5:
{'name': 'Sara', 'email': 'sara@example.com', 'phone': '555-123-4567', 'location': None, 'age': 22}


Extended Chat 6:
{'name': None, 'email': None, 'phone': None, 'location': None, 'age': None}


Extended Chat 7:
{'name': 'David', 'email': 'david@example.com', 'phone': None, 'location': 'Pune', 'age': None}


